In [1]:
#pip install vaderSentiment
#pip install emoji

#pip install autocorrect

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.parser import parse
import re

import emoji

from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet,stopwords
from nltk import word_tokenize,sent_tokenize
import nltk
from nltk.tokenize import regexp_tokenize 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from autocorrect import Speller

%matplotlib inline

In [3]:
nltk.download('words')
spell = Speller(lang='en')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package words to
[nltk_data]     /Users/anupprakash/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anupprakash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/anupprakash/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anupprakash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anupprakash/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
slangs = {}
slangs_df = pd.read_csv('/Users/anupprakash/Desktop/Review And Stock Price/FlipkartReviews/AllSlangs.csv')
slangs_df.drop('Unnamed: 0',1,inplace=True)
slangs_df.dropna(inplace=True)

In [5]:
for index,rows in slangs_df.iterrows():slangs[str(rows['Abbreviation'].replace(' ',''))] = str(rows['FullForm'].lower()).strip(' ')

In [6]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
"n't":'not' 
}

In [7]:
# Reading Excel Sheet from folder
#/content/drive/My Drive/ColabDataset/SentimentAndStockStudy/TawaReview.xlsx
BottomPressureCooker = pd.read_excel("/Users/anupprakash/Desktop/Review And Stock Price/FlipkartReviews/BottomPressureCooker.xlsx","AllReview")

In [8]:
BottomPressureCooker.head(10)#MixtureGrinderReview

,User Rating,User Review,User Name,Review Date
0,NaN,NaN,NaN,NaN
1,5.0,I have quite impressed with the amazing qualit...,Sushant Chaulkar,"Jun, 2019"
2,5.0,Sure Delivery from Flipkart not 5 🌟 I will gi...,Patrick Michael,5months ago
3,4.0,this price is very best for this product and a...,chetan singh,7months ago
4,4.0,it's very nice product so I'm happy and excite...,Lokanath Patra,"Jan, 2019"
5,5.0,Nice product. Worth of writing a review. Excel...,Sultan Mustafijul Hoque,"Nov, 2018"
6,5.0,super good work I like that's,Flipkart Customer,10months ago
7,4.0,i bought this to my wife.. she felt very happy...,Nagaraj Govindaraj,7months ago
8,5.0,good product worth for money pack is also good...,Flipkart Customer,8months ago
9,4.0,very nice super product but no ISI mark improv...,Ganesh Rk,8months ago


In [9]:
class PreprocessingData:

  def __init__(self,df,df_name='',location=''):
    self.df = df
    self.df.dropna(inplace=True)
    self.df_name = df_name
    self.location = location


  # ----------------------------- Change Raw date to Pandas Date Time foramt -----------------------------------------
  def ago_do_date(self,ago):
    j = re.search(r'(\d+)(\w+) ago',ago)
    if(j):
      value = j.group(1)
      unit = j.group(2)
      if(not unit.endswith("s")):
        unit+='s'
      delta = relativedelta(**{unit: int(value)})
      date = datetime.now() - delta
      return date.date()
    elif(ago=='Today'):
      return datetime.now().date()
    elif(ago!='Today'):
      dt = parse(ago.replace(",",''))
      return dt.date()

  

  # ----------------------------- Removing Emojis From Raw Text -----------------------------------------
  def removingEmojis(self,reviews):
    return ''.join(rev for rev in reviews if rev not in emoji.UNICODE_EMOJI)



  # ----------------------------- Removing Slangs from Raw Text -----------------------------------------
  def removingSlangs(self,reviews):
    doc = regexp_tokenize(reviews, "[\w']+") 

    for token in doc:
      if(token in CONTRACTION_MAP):
        reviews = reviews.replace(token,CONTRACTION_MAP[token])
      elif(token in slangs):
        reviews = reviews.replace(token,slangs[token])
    return reviews

  
  
  # ----------------------------- Checking Non-English Words and Replacing Miss-Pronounce Word With Actuall Words  -----------------------------------------
  def checkingNonEnglishWords(self,reviews):
    words = set(nltk.corpus.words.words())
    for w in regexp_tokenize(reviews, "[\w']+"):
      if ((w.lower() not in words) and (w.lower()!= 'flipkart')):
        reviews = reviews.replace(w,spell(w))
    return reviews


  
  # ----------------------------- Part of Speech Tagging -----------------------------------------
  def get_wordnet_pos(self,pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

        
  
  # ----------------------------- Removing StopWords and Lemmatizing Words -----------------------------------------
  def CleaningData(self,val):
    rex = re.sub(r'[^a-zA-Z0-9]+',' ',val)

    pos = pos_tag(word_tokenize(rex))

    filter = [WordNetLemmatizer().lemmatize(x[0],PreprocessingData.get_wordnet_pos(self,x[1])) for x in pos if x[0] not in stopwords.words('english')]

    filter = ' '.join(filter)

    return filter

    
  
  # ----------------------------- Getting the Polarity of Text(both Clean and Raw) -----------------------------------------
  def sentiment_analyzer_scores(self,sentence):
    analyser = SentimentIntensityAnalyzer()
    score = analyser.polarity_scores(sentence)
    if(score['compound']>=0.2):
        return [score['neg'],score['pos'],score['neu'],score['compound'],str('pos')]
    elif(score['compound']<=-0.2):
        return [score['neg'],score['pos'],score['neu'],score['compound'],str('neg')]
    else:
        return [score['neg'],score['pos'],score['neu'],score['compound'],str('neu')]
  

  def processRawData(self):
    #Date
    self.df['DateOfReview'] = self.df['Review Date'].apply(lambda x :PreprocessingData.ago_do_date(self,x))

    
    #Emojis
    self.df['User Review'] = self.df['User Review'].apply(lambda x :PreprocessingData.removingEmojis(self,x))

    #Removing Slangs
    self.df['User Review'] = self.df['User Review'].apply(lambda x :PreprocessingData.removingSlangs(self,x))

    #checkingNonEnglishWords
    self.df['User Review'] = self.df['User Review'].apply(lambda x :PreprocessingData.checkingNonEnglishWords(self,x))

    #CleaningData
    self.df['FinalText'] = self.df['User Review'].apply(lambda x :PreprocessingData.CleaningData(self,x))
    

    #sentiment_analyzer_scores
    (self.df['FinalTextSentimentNegative'], self.df['FinalTextSentimentPositive'], self.df['FinalTextSentimentNeutral'], self.df['FinalTextSentimentCompound'], self.df['FinalTextSentiment'])  = 0,0,0,0,0
    (self.df['UserReviewSentimentNegative'],self.df['UserReviewSentimentPositive'],self.df['UserReviewSentimentNeutral'],self.df['UserReviewSentimentCompound'],self.df['UserReviewSentiment']) =0,0,0,0,0
    m = self.df['FinalText'].apply(lambda x :PreprocessingData.sentiment_analyzer_scores(self,x))    
    j = self.df['User Review'].apply(lambda x :PreprocessingData.sentiment_analyzer_scores(self,x))
    for x in range(len(m)):
        self.df['FinalTextSentimentNegative'].iloc[x] = m.iloc[x][0]
        self.df['FinalTextSentimentPositive'].iloc[x]= m.iloc[x][1]
        self.df['FinalTextSentimentNeutral'].iloc[x]= m.iloc[x][2]
        self.df['FinalTextSentimentCompound'].iloc[x] = m.iloc[x][3] 
        self.df['FinalTextSentiment'].loc[x] = m.iloc[x][4]

    for x in range(len(j)):
        self.df['UserReviewSentimentNegative'].iloc[x] = j.iloc[x][0]
        self.df['UserReviewSentimentPositive'].iloc[x]= j.iloc[x][1]
        self.df['UserReviewSentimentNeutral'].iloc[x]= j.iloc[x][2]
        self.df['UserReviewSentimentCompound'].iloc[x] = j.iloc[x][3] 
        self.df['UserReviewSentiment'].loc[x] = j.iloc[x][4]
        #print(type(m))
    
    self.df.to_csv(str(fileLocation+excelSheetName+'.csv'),index=False)



fileLocation = '/Users/anupprakash/Desktop/Review And Stock Price/FlipkartReviews/PreProcessDataset/'
excelSheetName = "BottomPressureCooker"
dataFrame = BottomPressureCooker


mj = PreprocessingData(dataFrame ,excelSheetName,fileLocation)
mj.processRawData()



BottomPressureCooker

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,User Rating,User Review,User Name,Review Date,DateOfReview,FinalText,FinalTextSentimentNegative,FinalTextSentimentPositive,FinalTextSentimentNeutral,FinalTextSentimentCompound,FinalTextSentiment,UserReviewSentimentNegative,UserReviewSentimentPositive,UserReviewSentimentNeutral,UserReviewSentimentCompound,UserReviewSentiment
1,5.0,I have quite impressed with the amazing qualit...,Sushant Chaulkar,"Jun, 2019",2019-06-03,I quite impress amazing quality pressure cooke...,0.121,0.440,0.439,0.9236,pos,0.053,0.276,0.670,0.9430,pos
2,5.0,Youre Delivery from Flipkart not 5 I will gi...,Patrick Michael,5months ago,2020-03-03,Youre Delivery Flipkart 5 I give 100 Butter fl...,0.000,0.440,0.560,0.8957,pos,0.000,0.289,0.711,0.8957,pos
3,4.0,this price is very best for this product and a...,chetan singh,7months ago,2020-01-03,price best product valuable company trust good,0.000,0.818,0.182,0.9260,pos,0.000,0.520,0.480,0.9335,pos
4,4.0,it is very nice product so I am happy and exci...,Lokanath Patra,"Jan, 2019",2019-01-03,nice product I happy excite price thank Flipkart,0.000,0.752,0.248,0.9022,pos,0.000,0.479,0.521,0.8991,pos
5,5.0,Nice product. Worth of writing a review. Excel...,Sultan Mustafijul Hoque,"Nov, 2018",2018-11-03,Nice product Worth write review Excellent deli...,0.000,0.548,0.452,0.8720,pos,0.000,0.456,0.544,0.8720,pos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,4.0,Good,Muhammed Farhan Mammu,1month ago,2020-07-03,Good,0.000,1.000,0.000,0.4404,pos,0.000,1.000,0.000,0.4404,pos
1038,4.0,Excellent,Vanishree Yeshwanth,1month ago,2020-07-03,Excellent,0.000,1.000,0.000,0.5719,pos,0.000,1.000,0.000,0.5719,pos
1039,5.0,Excellent,salauddin Biswas,1month ago,2020-07-03,Excellent,0.000,1.000,0.000,0.5719,pos,0.000,1.000,0.000,0.5719,pos
1040,5.0,very Best product this is good product thanks...,S K Tiwari,1month ago,2020-07-03,Best product good product thanks Flipkart product,0.000,0.714,0.286,0.8750,pos,0.000,0.563,0.437,0.8832,pos
